In [1]:
# string output parser

In [ ]:
#without string output parser
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate

load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

# 1st prompt -> detailed report
template1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

# 2nd prompt -> summary
template2 = PromptTemplate(
    template='Write a 5 line summary on the following text. /n {text}',
    input_variables=['text']
)

prompt1 = template1.invoke({'topic':'black hole'})

result = model.invoke(prompt1)

prompt2 = template2.invoke({'text':result.content})

result1 = model.invoke(prompt2)

print(result1.content)


/Users/daniyalkhan/anaconda3/envs/genAI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Here's a 5-line summary:

Black holes are regions of extreme gravity where even light becomes trapped. Formed from collapsed massive stars, they exert a powerful influence on surrounding matter and galaxies, playing a crucial role in cosmic processes.  Despite their gravitational dominance, black holes remain shrouded in secrecy with much about their nature still unknown, prompting ongoing research.  We observe them indirectly through the emission of light and gravitational waves, providing valuable insight into their behaviors. Black hole discovery pushes the boundaries of our scientific understanding and challenges our understanding of the cosmos. 

Let me know if you'd like any further elaboration or a different style of summary. 



In [3]:
#with string output parser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()


model = ChatOpenAI()

# 1st prompt -> detailed report
template1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

# 2nd prompt -> summary
template2 = PromptTemplate(
    template='Write a 5 line summary on the following text. /n {text}',
    input_variables=['text']
)

parser = StrOutputParser()

chain = template1 | model | parser | template2 | model | parser

result = chain.invoke({'topic':'black hole'})

print(result)

Black holes are regions in space with strong gravitational pull where not even light can escape, making them mysterious objects. They are formed when massive stars collapse under their own gravity or through the merger of neutron stars or black holes. Black holes have properties like mass, electric charge, and angular momentum, with different types including stellar, supermassive, and intermediate black holes. Studying black holes is crucial in astrophysics as they provide insights into the nature of spacetime, the formation and evolution of galaxies, and the birth and death of stars. Overall, black holes are enigmatic objects that challenge our understanding of the universe and help expand our knowledge of the cosmos.


In [1]:
#json output parser

In [2]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

load_dotenv()

# Define the model
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

parser = JsonOutputParser()

template = PromptTemplate(
    template='Give me 5 facts about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)


/Users/daniyalkhan/anaconda3/envs/genAI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
prompt= template.format(topic= "Living Water")
print(prompt)
result= model.invoke(prompt)
final_result= parser.parse(result.content)
print(final_result)

Give me 5 facts about Living Water 
 Return a JSON object.
[{'fact': 'Living Water is a non-denominational Christian ministry.', 'details': 'Founded in the 1980s, Living Water is known for its focus on empowering individuals through the life-transformative teachings of Jesus Christ.'}, {'fact': 'Living Water emphasizes the importance of personal relationship with God.', 'details': 'The ministry aims to help individuals develop a deep and abiding connection with God through prayer, scripture, and spiritual guidance.'}, {'fact': 'Living Water conducts outreach programs and discipleship activities within communities.', 'details': 'To spread the message of love and salvation, Living Water engages in community service, offers workshops, and provides support to those in need.'}, {'fact': 'The Living Water movement respects all people and promotes reconciliation and unity.', 'details': 'The practice of spiritual awareness and meditation forms a shared understanding among individuals, regardle

In [9]:
chain = template | model | parser #can also form a chain
result = chain.invoke({'topic':'black hole'})
print(result)

[{'fact': 'Black holes are regions of spacetime where gravity is so strong that nothing, not even light, can escape.', 'category': 'Characteristics'}, {'fact': 'Black holes form when massive stars collapse at the end of their lifespans.', 'category': 'Origin'}, {'fact': 'There are various types of black holes, fromStellar-mass black holes, which are formed from the collapses of individual stars, to supermassive black holes, which reside at the centers of most galaxies.', 'category': 'Types'}, {'fact': 'The event horizon of a black hole is the point of no return, beyond which the curvature of spacetime becomes so extreme that escape is impossible.', 'category': 'Boundaries'}, {'fact': 'Black holes are not black, they emit radiation called Hawking radiation, as particles constantly interact near the event horizon.', 'category': 'Physics'}]


In [10]:
#structured output parser

In [11]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

load_dotenv()

# Define the model
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='Fact 3 about the topic'),
]

parser = StructuredOutputParser.from_response_schemas(schema)

template = PromptTemplate(
    template='Give 3 fact about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

result = chain.invoke({'topic':'black hole'})

print(result)

{'fact_1': 'Black holes are regions of spacetime where gravity is so strong that nothing, not even light, can escape.', 'fact_2': 'The immense gravity of a black hole comes from the large amount of mass compressed into a tiny space.', 'fact_3': "While we can't see black holes directly, we can detect them through their impact on surrounding matter, such as how stars orbit them."}


In [13]:
prompt= template.format(topic= 'Living Water')
print(prompt)
result= model.invoke(prompt)
final_result= parser.parse(result.content)
print(final_result)

Give 3 fact about Living Water 
 The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"fact_1": string  // Fact 1 about the topic
	"fact_2": string  // Fact 2 about the topic
	"fact_3": string  // Fact 3 about the topic
}
```
{'fact_1': 'Living Water is a band known for its blend of acoustic folk and alternative rock genres.', 'fact_2': 'They are a Wisconsin-based band.', 'fact_3': 'They are known for their strong, emotional lyrics and captivating live performances.'}


In [14]:
#pydantic output parser

In [15]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()

# Define the model
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

class Person(BaseModel):

    name: str = Field(description='Name of the person')
    age: int = Field(gt=18, description='Age of the person')
    city: str = Field(description='Name of the city the person belongs to')

parser = PydanticOutputParser(pydantic_object=Person)

template = PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

In [16]:
prompt= template.invoke({'place': 'Chinese'})
print(prompt)
result= model.invoke(prompt)
final_result= parser.parse(result.content)
print(final_result)

text='Generate the name, age and city of a fictional Chinese person \n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "Name of the person", "title": "Name", "type": "string"}, "age": {"description": "Age of the person", "exclusiveMinimum": 18, "title": "Age", "type": "integer"}, "city": {"description": "Name of the city the person belongs to", "title": "City", "type": "string"}}, "required": ["name", "age", "city"]}\n```'
name='Lin Xiao' age=32 city='Shanghai'


In [ ]:

chain = template | model | parser

final_result = chain.invoke({'place':'sri lankan'})

print(final_result)